# Fonctions utiles

In [ ]:
import numpy as np
import pandas as pd
import random
from itertools import combinations

In [ ]:
def convert_attributes_into_pandas(filename):
    return pd.read_csv(filename)

In [ ]:
df_attributes = convert_attributes_into_pandas("attributes_data.csv")
df_attributes

In [ ]:
df_test = convert_attributes_into_pandas("test_attributes.csv")
type(df_test)

In [ ]:
indexes = df_attributes.columns
indexes

Réduction matrice

In [ ]:
#On souhaite pouvoir réduire la matrice selon certaines colonnes, dont les attributs correspondants seront fixés
#et ne pourront donc pas changer pendant l'algorithme génétique.
#Pour cela on va récupérer une liste d'attributs avec certaines valeurs et utiliser cette liste pour réduire la matrice

fixed_attributes = {"Pale_Skin":-1}

In [ ]:
def matrix_reduction(df_attributes,fixed_attributes):
    attributes = list(fixed_attributes.keys())
    values = list(fixed_attributes.values())
    for i in range(len(attributes)):
        df_attributes.drop(df_attributes.index[df_attributes[attributes[i]]!= values[i]],inplace=True)
    return df_attributes.index.tolist()


In [ ]:
matrix_reduction(df_attributes,fixed_attributes)

In [ ]:
def get_attributes_from_ID(df_attributes,photo_id):
    return pd.DataFrame(df_attributes.loc[df_attributes['ID'] == photo_id])

In [ ]:
get_attributes_from_ID(df_attributes,'043013.jpg')

In [ ]:
def delete_photos(df_attributes,photos_ids):
    for i in range(len(photos_ids)):
        df_attributes.drop(df_attributes[df_attributes.ID == str(photos_ids[i])].index,inplace=True)
    return df_attributes

## Algorithme génetique

In [ ]:
def randomly_choose_photos(df_attributes,n):
    list_ID = df_attributes.index.tolist()
    return random.sample(list_ID,n)  

In [ ]:
randomly_choose_photos(df_attributes,4)

In [ ]:
encoded_imgs= np.load('encoded_imgs.npy')

In [ ]:
def crossover_pixels(parents,pc):
    index_parents = list(range(0,len(parents)))
    p_combinations = list(combinations(index_parents,2))
    offsprings= []
    for i in range(len(p_combinations)):
        parent_1 = (parents[p_combinations[i][0]]).tolist()
        parent_2 = (parents[p_combinations[i][1]]).tolist()
        k = int(parents[0].size*pc)  #proportion pc from parent1
        offsprings.append(random.sample(parent_1,k)+ random.sample(parent_2,parents[0].size-k))
    return offsprings

In [ ]:
def crossover_attributes(parent1,parent2,pc):
    list1 = parent1.columns.tolist()
    list1.remove('ID')
    #split of attributes list into two based on proportions
    k = int(len(list1)*pc)   #proportion pc from parent1
    l1 = random.sample(list1,k)
    l2 = []
    for attribute in list1 : 
        if(attribute not in l1):
            l2.append(attribute)
    attributes = l1 + l2 
    #getting each attribute respective value
    values = []
    for i in range(len(l1)):
        values.append(int(parent1.loc[:,l1[i]]))
    for j in range(len(l2)):
        values.append(int(parent1.loc[:,l2[j]]))
    
    #creating offspring as dataframe
    return pd.DataFrame(values,attributes).transpose()

In [ ]:
offsprings = crossover_pixels(encoded_imgs,0.3)

In [ ]:
def mutation_pixels(parents,pm):
    muted_parents = parents
    for j in range(len(parents)): 
        for i in range(parents[0].size) :
            r = random.random()
            if r < pm :
                muted_parents[j][i] = random.randint(0, 255)
    return muted_parents

In [ ]:
def mutation_attributes(offspring,pm):
    muted_offspring = offspring
    for i in range(offspring.shape[1]) :
        r = random.random()
        if r < pm :
            muted_offspring.iloc[0,i] = offspring.iloc[0,i] * (-1)
    return muted_offspring